In [14]:
import tensorflow as tf
import keras
from keras import layers
import make_val_prepare_data_aclImdb as imdb_data

max_length = 600
max_tokens = 10000

text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    output_sequence_length=max_length
)

text_only_train_ds = imdb_data.train_ds.map(lambda x,y:x)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = imdb_data.train_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_val_ds = imdb_data.val_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
int_test_ds = imdb_data.test_ds.map(
    lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)


2024-12-20 23:42:30.499947: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [18]:
import transformer_encoder

vocab_size = 2000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None, ), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim)(inputs)
x = transformer_encoder.TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer = "rmsprop",
             loss="binary_crossentropy",
             metrics = ["accuracy"])

model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding_4 (Embedding)         │ (None, None, 256)      │       512,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_encoder_4           │ (None, None, 256)      │       543,776 │
│ (TransformerEncoder)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d_4          │ (None, 256)            │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,056,033 (4.03 MB)

 Trainable params: 1,056,033 (4.03 MB)

 Non-trainable params: 0 (0.00 B)

In [19]:
callbacks = [
    keras.callbacks.ModelCheckpoint("transformer_encoder.keras",save_best_only=True),
    keras.callbacks.TensorBoard(log_dir="transformer_encoder")]

model.fit(int_train_ds, validation_data=int_val_ds, epochs=20,
         callbacks=callbacks)

Epoch 1/20


I0000 00:00:1734709709.387096   25110 service.cc:146] XLA service 0x7f2fd40129b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734709709.387176   25110 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 SUPER, Compute Capability 7.5
2024-12-20 23:48:29.435432: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-20 23:48:29.665388: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90100


  1/704 ━━━━━━━━━━━━━━━━━━━━ 1:49:07 9s/step - accuracy: 0.6250 - loss: 1.5468

I0000 00:00:1734709716.312197   25110 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


704/704 ━━━━━━━━━━━━━━━━━━━━ 118s 155ms/step - accuracy: 0.6186 - loss: 0.7960 - val_accuracy: 0.7748 - val_loss: 0.4674
Epoch 2/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 105s 150ms/step - accuracy: 0.8259 - loss: 0.3969 - val_accuracy: 0.6552 - val_loss: 0.7882
Epoch 3/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 141s 148ms/step - accuracy: 0.8434 - loss: 0.3574 - val_accuracy: 0.8284 - val_loss: 0.3852
Epoch 4/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 106s 150ms/step - accuracy: 0.8504 - loss: 0.3421 - val_accuracy: 0.7868 - val_loss: 0.4623
Epoch 5/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 105s 150ms/step - accuracy: 0.8557 - loss: 0.3351 - val_accuracy: 0.8128 - val_loss: 0.4207
Epoch 6/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 104s 148ms/step - accuracy: 0.8594 - loss: 0.3277 - val_accuracy: 0.7844 - val_loss: 0.4658
Epoch 7/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 106s 151ms/step - accuracy: 0.8631 - loss: 0.3174 - val_accuracy: 0.7832 - val_loss: 0.4929
Epoch 8/20
704/704 ━━━━━━━━━━━━━━━━━━━━ 107s 152ms/step - accuracy: 0.8687 - loss: 0.30

In [20]:
model = keras.models.load_model("transformer_encoder.keras",
                                custom_objects={
                                   "transformer_encoder": transformer_encoder.TransformerEncoder})

print(f"test acc: {model.evaluate(int_test_ds)[1]:.3f}")

/home/binbin/anaconda3/envs/dl/lib/python3.10/site-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'transformer_encoder_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


782/782 ━━━━━━━━━━━━━━━━━━━━ 50s 63ms/step - accuracy: 0.8501 - loss: 0.3399
test acc: 0.850
